This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [5]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'c_ZHsdwMRyyHsswSxjHB'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [18]:
# First, import the relevant modules

from urllib import request
import json




In [19]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

rsp = request.urlopen(f'https://data.nasdaq.com/api/v3/datasets/HKEX/06669/data.json?api_key={API_KEY}')

body = json.loads(rsp.read().decode('utf8'))

dataset = body['dataset_data']

column_names = dataset['column_names']

# enforce that the column names have not changed
assert(column_names == [
    'Date',
    'Nominal Price',
    'Net Change',
    'Change (%)',
    'Bid',
    'Ask',
    'P/E(x)',
    'High',
    'Low',
    'Previous Close',
    'Share Volume (000)',
    'Turnover (000)',
    'Lot Size'
])

In [25]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

dataset

{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Nominal Price',
  'Net Change',
  'Change (%)',
  'Bid',
  'Ask',
  'P/E(x)',
  'High',
  'Low',
  'Previous Close',
  'Share Volume (000)',
  'Turnover (000)',
  'Lot Size'],
 'start_date': '2021-08-24',
 'end_date': '2021-09-17',
 'frequency': None,
 'data': [['2021-09-17',
   14.66,
   None,
   None,
   14.66,
   14.84,
   None,
   15.0,
   14.42,
   14.8,
   770.0,
   11253.0,
   None],
  ['2021-09-16',
   14.8,
   None,
   None,
   14.8,
   14.82,
   None,
   15.68,
   14.46,
   15.4,
   1409.0,
   21086.0,
   None],
  ['2021-09-15',
   15.4,
   None,
   None,
   15.3,
   15.4,
   None,
   16.48,
   15.28,
   16.4,
   1067.0,
   16830.0,
   None],
  ['2021-09-14',
   16.4,
   None,
   None,
   16.36,
   16.4,
   None,
   16.72,
   16.24,
   16.72,
   1258.0,
   20723.0,
   None],
  ['2021-09-13',
   16.72,
   None,
   None,
   16.7,
   16.74,
   None,
   17.54,
   16.68,
   17.56,
   1356.0,
  

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [44]:
# Helpers

def column_array(name):
    index = column_names.index(name)
    
    return [row[index] for row in dataset['data']]



In [30]:
from functools import reduce

# 3) Highest and lowest opening prices
assert(0 < len(dataset['data']))

# Concatenate date, and previous close columns
prices = zip(
    column_array('Date'), 
    column_array('Previous Close')
)

# Remove None
no_none = filter(lambda x: None not in x, prices)

# Helper function to simultaneously find the min and max values
# while carrying their dates
def separate(min_max, item):
    if None is min_max:
        return (item, item)
    else:
        min_item, max_item = min_max
    
        return (
            min(min_item, item, key=lambda x: x[1]),
            max(max_item, item, key=lambda x: x[1]))
    

# Apply helper function to find min and max open prices
(min_date, min_price), (max_date, max_price) = reduce(separate, no_none, None)

print(f'lowest  opening price: {min_price}, {min_date}')
print(f'highest opening price: {max_price}, {max_date}')


lowest  opening price: 14.8, 2021-09-17
highest opening price: 18.86, 2021-08-26


In [32]:
# 4) What was the largest change in any one day (based on High and Low price)?

# verticaly concatenate date, high and low columns, and remove rows
# that have None
prices = filter(
    lambda x: None not in x, 
    zip(
        column_array('Date'), 
        column_array('High'), 
        column_array('Low')
    )
)


# Compute the price range
prices = (
    (date, high-low) for date, high, low in prices
)

date, largest_change = max(prices, key=lambda x: x[1])

print(f'largest price change of {largest_change} occured on {date}')


largest price change of 1.9400000000000013 occured on 2021-08-25


In [45]:
# 5) What was the largest change between any two days (based on Closing Price)?

import typing

prices = filter(
    lambda x: None not in x, 
    zip(
        column_array('Date'), 
        column_array('Previous Close')
    )
)


def largest_change_func(acc, current_item):

    class Acc(typing.NamedTuple):
        last_item: typing.Tuple[str, float] 
        largest_change: typing.Tuple[str, float]


    if None is acc:
        return Acc(current_item, None)
    else:
        change = current_item[1] - acc.last_item[1]
        
        #print('change: %f %s'%(change, current_item[0]))
        
        if None is acc.largest_change or \
           abs(change) > abs(acc.largest_change[1]): 
            return Acc(current_item, (current_item[0], change))
        else:
            return Acc(current_item, acc.largest_change)

acc = reduce(largest_change_func, prices, None)

date, price_change = acc.largest_change

print(f'largest change b/w two days was {price_change} on {date}')



largest change b/w two days was -1.259999999999998 on 2021-08-25


In [49]:
# 6) What was the average daily trading volume during this year?

volume = list(
    filter(
        lambda x: None is not x, 
        column_array(
            'Share Volume (000)'
        )
    )
)


avg = sum(volume) / len(volume)

print(f'average daily trading volume was: {avg}')


average daily trading volume was: 3341.6315789473683
